<a href="https://colab.research.google.com/github/barzansaeedpour/advanced-python-notebooks/blob/main/SOLID_Principles_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOLID Principles with Python

SOLID principles are a series of good practices that will make you write better quality software. This looks like a long article but actually it is not. Just the codes takes a little bit more space :)

SOLID stands for:

- **S**: Single responsibility principle
- **O**: Open/Closed principle
- **L**: Liskov’s substitution principle
- **I**: Interface segregation principle
- **D**: Dependency inversion principle

## Resources

- https://medium.com/@m.nusret.ozates/solid-principles-with-python-245e45f9b1f8